In [ ]:
import os
import datetime

In [ ]:
class Downloader:
    def __init__(self):
        self.tar_path = '/home/kinoshita/synthetic/tar'
        self.bin_path = '/mnt/hgfs/kagoshima/bin'  # Seminarをマウントしている
        %mkdir -p $self.tar_path
        self.delete_directory(tar_flag=True)
        %mkdir -p $self.tar_path
        %mkdir -p $self.bin_path
        %cd ~/synthetic/tar
    
    def set_date(self, dt):
        # URLを指定
        directory = 'http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/jma-radar/synthetic/original'
        date = dt.strftime('%Y/%m/%d')
        self.timestamp = dt.strftime('%Y%m%d%H%M%S')
        self.tar_filename = 'Z__C_RJTD_'+self.timestamp+'_RDR_JMAGPV__grib2.tar'
        self.URL = directory +'/'+ date +'/'+ self.tar_filename
        
    def get_bin_file(self):
        # binファイルがあるかを確認
        bin_file_path = self.bin_path+'/Z__C_RJTD_'+self.timestamp+'_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin'
        if os.path.exists(bin_file_path):
            print('already exist: ',bin_file_path)
        else:
            # tarファイルを~/tarに保存
            _ = !wget -nc -P $self.tar_path $self.URL
            # tarファイルから1kmメッシュデータのみを取り出す
            filelist = !tar -tf $self.tar_filename  # tarファイル内ファイル名リスト
            ggis_name = [s for s in filelist if 'Ggis1km' in s][0]  # ファイル名を取得
            !tar -C $self.bin_path -xvf $self.tar_filename $ggis_name # tarファイルから1kmメッシュ全国合成レーダーエコー強度GPVのみを/binに取り出す
            !rm $self.tar_filename  # tarファイルは不要なので削除
    
    def delete_directory(self, tar_flag=False, bin_flag=False):
        if tar_flag:
            !rm -r $self.tar_path
        if bin_flag:
            !rm -r $self.bin_path

In [ ]:
start = '2017/01/01 00:00:00'
end = '2018/01/01 00:00:00'
dt = datetime.datetime.strptime(start, '%Y/%m/%d %H:%M:%S')
dt_limit = datetime.datetime.strptime(end, '%Y/%m/%d %H:%M:%S')

downloader = Downloader()
while dt < dt_limit:
    downloader.set_date(dt)
    downloader.get_bin_file()
    dt = dt + datetime.timedelta(minutes=10)